In [ ]:
## Comment out in Kaggle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install annoy
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=550699 sha256=96e3baa87d29e45749ca25f0afe4fe263ad258fc5901372f5fbe16c753e3ed64
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86

In [ ]:
import json
import pandas as pd
import numpy as np

from annoy import AnnoyIndex

# Load data

In [ ]:
cols = ['id', 'authors', 'doi', 'title', 'abstract']
data = []
file_name = '/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json'
#file_name = '/content/drive/MyDrive/VT/Information Storage Project Files/arxiv-metadata-oai-snapshot.json'

with open(file_name, encoding='latin-1') as f:
    for line in f:
        doc = json.loads(line)
        lst = [doc['id'], doc['authors'], doc['doi'], doc['title'], doc['abstract']]
        data.append(lst)

df_data = pd.DataFrame(data=data, columns=cols)

print(df_data.shape)

df_data.head()

(2440876, 5)


,id,authors,doi,title,abstract
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,0704.0002,Ileana Streinu and Louis Theran,None,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,Hongjun Pan,None,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,0704.0004,David Callan,None,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,0704.0005,Wael Abu-Shammala and Alberto Torchinsky,None,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...


In [ ]:
df_data['prepared_text'] = '{title}: ' + df_data['title'] + ' {abstract}: ' + df_data['abstract']
df_data.head()

,id,authors,doi,title,abstract,prepared_text
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,{title}: Calculation of prompt diphoton produc...
1,0704.0002,Ileana Streinu and Louis Theran,None,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",{title}: Sparsity-certifying Graph Decompositi...
2,0704.0003,Hongjun Pan,None,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,{title}: The evolution of the Earth-Moon syste...
3,0704.0004,David Callan,None,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,{title}: A determinant of Stirling cycle numbe...
4,0704.0005,Wael Abu-Shammala and Alberto Torchinsky,None,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,{title}: From dyadic $\Lambda_{\alpha}$ to $\L...


In [ ]:
df_data['prepared_text'][0]

'{title}: Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies {abstract}:   A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the 

# Embed data (Option B - Sentence Transformer)
This will embedd the same approach as above and save the same way, only run one of the two options

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("thenlper/gte-base")
model.max_seq_length = 512

import time
start = time.time()

embeds = model.encode(list(df_data['prepared_text'][:100000]), batch_size=256, show_progress_bar=True)

end = time.time()
print(end-start)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

459.9779119491577


In [ ]:
embeds.shape

(100000, 768)

# Use Annoy to store the embeddings

In [ ]:
start = time.time()

annoy_model = AnnoyIndex(embeds.shape[1], metric='angular')
for i, embed in enumerate(embeds):
    annoy_model.add_item(i, embed)

annoy_model.build(500)

end = time.time()
print(end-start)


34.008646726608276


In [ ]:
annoy_model.save('/content/drive/MyDrive/gte-base_annoy_100000_tree_500.ann')

True

# Load Annoy indices

In [ ]:
annoy_model_load = AnnoyIndex(768, metric='angular')
annoy_model_load.load("/content/drive/MyDrive/VT/Information Storage Project Files/Complete pipeline/gte_embeds.ann", prefault=True)

print(annoy_model_load.get_n_items())
print(annoy_model_load.get_n_trees())


500
997


# Get top k results (Option B - Sentence Transformers)

In [ ]:
def get_top_results(query, k):
  input_type_query = "search_query"

  query_embed = model.encode(query)

  similar_item_ids = annoy_model_load.get_nns_by_vector(query_embed, k, include_distances=True)

  query_results = pd.DataFrame(data={'id': df_data.iloc[similar_item_ids[0]]['id'],
                                     'texts': df_data.iloc[similar_item_ids[0]]['prepared_text'],
                                     'authors': df_data.iloc[similar_item_ids[0]]['authors'],
                                     'doi': df_data.iloc[similar_item_ids[0]]['doi'],
                                     'distance': similar_item_ids[1]})

  return query_results

In [ ]:
get_top_results("quantum computer", 10)

,id,texts,authors,doi,distance
201,0704.0202,{title}: Towards Minimal Resources of Measurem...,Simon Perdrix,10.1088/1367-2630/9/6/206,0.800560
322,0704.0323,{title}: General sequential quantum cloning {a...,"Gui-Fang Dang, and Heng Fan",10.1088/1751-8113/41/15/155303,0.825306
41,0704.0042,"{title}: General System theory, Like-Quantum S...",Ignazio Licata,None,0.842215
481,0704.0482,{title}: Implementation of holonomic quantum c...,"Zhang-qi Yin, Fu-li Li, Peng Peng",10.1103/PhysRevA.76.062311,0.843863
33,0704.0034,{title}: Origin of adaptive mutants: a quantum...,Vasily Ogryzko,None,0.844965
267,0704.0268,{title}: Automated Generation of Layout and Co...,"Mark Whitney, Nemanja Isailovic, Yatish Patel,...",None,0.845553
408,0704.0409,{title}: On the over-barrier reflection in qua...,"D.G. Levkov, A.G. Panin, S.M. Sibiryakov",10.1103/PhysRevA.76.032114,0.868117
277,0704.0278,{title}: q-Deformed spin foam models of quantu...,"Igor Khavkine, J. Daniel Christensen",10.1088/0264-9381/24/13/009,0.868165
419,0704.0420,{title}: The Hourglass - Consequences of Pure ...,Donald McCartor,None,0.873012
45,0704.0046,{title}: A limit relation for entropy and chan...,"I. Csiszar, F. Hiai and D. Petz",10.1063/1.2779138,0.880496


# Evaluation

In [ ]:
eval_data = pd.read_csv('/content/drive/MyDrive/VT/Information Storage Project Files/evaluation_data_500.csv')

In [ ]:
eval_data.head()

,Unnamed: 0,id,authors,doi,title,abstract,question1,question2
0,0,704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,- Calculation of diphoton production at hadron...,- Comparison of diphoton production from Higgs...
1,1,704.0002,Ileana Streinu and Louis Theran,NaN,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...","- How to characterize $(k,\ell)$-sparse graphs?","- How does the $(k,\ell)$-pebble game with col..."
2,2,704.0003,Hongjun Pan,NaN,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,- How has the Earth-Moon system evolved accord...,- What is the significance of the Roche's limi...
3,3,704.0004,David Callan,NaN,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,- How can acyclic single-source automata be en...,- What is the significance of Stirling cycle n...
4,4,704.0005,Wael Abu-Shammala and Alberto Torchinsky,NaN,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,- How to compute the $\Lambda_{\alpha}$ norm u...,- How are the Hardy spaces $H^p(R^N)$ describe...


## top k = 1 results

In [ ]:
k =  1
true_false = []
for i in range(len(eval_data)):
  question = eval_data['question1'][i]
  correct_answer = eval_data['id'][i]
  results = get_top_results(question, k)
  result_ids = [float(j) for j in results['id'].values]
  # print(correct_answer)
  # print(result_ids)
  true_false.append(correct_answer in result_ids)
  print(i)

for i in range(len(eval_data)):
  try:
    question = eval_data['question2'][i]
    correct_answer = eval_data['id'][i]
    results = get_top_results(question, k)
    result_ids = [float(j) for j in results['id'].values]
    # print(correct_answer)
    # print(result_ids)
    true_false.append(correct_answer in result_ids)
    print(i)
  except:
    print(i,  " - error")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ERROR:tornado.access:500 POST /v1beta/models/embedding-001:embedContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.67ms


156  - error
157
158  - error
159
160
161
162
163  - error
164
165
166
167
168
169
170  - error
171
172
173  - error
174
175  - error
176
177
178
179
180
181  - error
182
183
184
185
186
187
188
189
190
191  - error
192
193
194
195  - error
196
197  - error
198
199
200
201
202
203
204  - error
205
206
207
208
209
210
211
212
213
214
215  - error
216
217
218  - error
219  - error
220
221
222
223
224
225
226
227
228
229
230
231
232  - error
233
234
235
236
237
238
239
240  - error
241
242
243
244
245  - error
246
247
248
249
250
251
252
253
254
255
256
257
258
259  - error
260
261  - error
262
263
264  - error
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286  - error
287
288
289
290
291
292
293
294
295
296  - error
297
298
299
300  - error
301
302
303  - error
304
305  - error
306
307
308
309
310
311
312
313  - error
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333  - error
334
335
336
337  - error
338
339
340
341
342


In [ ]:
all(true_false)

False

In [ ]:
sum(true_false) / len(true_false)

0.9914438502673797

## top k = 3 results

In [ ]:
k =  3
true_false = []
for i in range(len(eval_data)):
  question = eval_data['question1'][i]
  correct_answer = eval_data['id'][i]
  results = get_top_results(question, k)
  result_ids = [float(j) for j in results['id'].values]
  # print(correct_answer)
  # print(result_ids)
  true_false.append(correct_answer in result_ids)
  print(i)

for i in range(len(eval_data)):
  try:
    question = eval_data['question2'][i]
    correct_answer = eval_data['id'][i]
    results = get_top_results(question, k)
    result_ids = [float(j) for j in results['id'].values]
    # print(correct_answer)
    # print(result_ids)
    true_false.append(correct_answer in result_ids)
    print(i)
  except:
    print(i,  " - error")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
all(true_false)

False

In [ ]:
sum(true_false) / len(true_false)

0.9989316239316239

In [ ]:
len(true_false)

936

## top k = 5 results

In [ ]:
k =  5
true_false = []
for i in range(len(eval_data)):
  question = eval_data['question1'][i]
  correct_answer = eval_data['id'][i]
  results = get_top_results(question, k)
  result_ids = [float(j) for j in results['id'].values]
  # print(correct_answer)
  # print(result_ids)
  true_false.append(correct_answer in result_ids)
  print(i)

for i in range(len(eval_data)):
  try:
    question = eval_data['question2'][i]
    correct_answer = eval_data['id'][i]
    results = get_top_results(question, k)
    result_ids = [float(j) for j in results['id'].values]
    # print(correct_answer)
    # print(result_ids)
    true_false.append(correct_answer in result_ids)
    print(i)
  except:
    print(i,  " - error")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
all(true_false)

True

In [ ]:
sum(true_false) / len(true_false)

1.0